This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [2]:
# get api key from your .env file
import os
from dotenv import load_dotenv  # if missing this module, simply run `pip install python-dotenv`

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

print(API_KEY)

dL7YWhFXv-QyqpN_Drjh


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [3]:
# First, import the relevant modules
import requests

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [36]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
dataset_params = {'database_code': 'FSE',
                  'dataset_code': 'AFX_X',
                  'return_format': 'json'}

url = 'https://data.nasdaq.com/api/v3/datasets/{database_code}/{dataset_code}/data.{return_format}'.format(**dataset_params)
# note, cannot grab data past 2020-12-01
r = requests.get(url, params={'api_key': API_KEY, 'start_date':'2020-11-29'})
print(r.url)
print('status: %i' %r.status_code)

https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?api_key=dL7YWhFXv-QyqpN_Drjh&start_date=2020-11-29
status: 200


In [38]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(r.text)

{"dataset_data":{"limit":null,"transform":null,"column_index":null,"column_names":["Date","Open","High","Low","Close","Change","Traded Volume","Turnover","Last Price of the Day","Daily Traded Units","Daily Turnover"],"start_date":"2020-11-29","end_date":"2020-12-01","frequency":"daily","data":[["2020-12-01",112.2,112.2,111.5,112.0,null,51.0,5703.0,null,null,null],["2020-11-30",111.0,113.6,111.0,112.1,null,315.0,35111.5,null,null,null]],"collapse":null,"order":null}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [81]:
#1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
# build url with exchange, ticker, format
dataset_params = {'database_code': 'FSE',
                  'dataset_code': 'AFX_X',
                  'return_format': 'json'}
url = 'https://data.nasdaq.com/api/v3/datasets/{database_code}/{dataset_code}/data.{return_format}'.format(**dataset_params)

# request data for year of 2017
request_params = {
    'api_key'       : API_KEY,
    'start_date'    : '2017-01-01',
    'end_date'      : '2017-12-31'
}
r = requests.get(url, params=request_params)
print(r.url)
print('status: %i' %r.status_code)

https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?api_key=dL7YWhFXv-QyqpN_Drjh&start_date=2017-01-01&end_date=2017-12-31
status: 200


In [131]:
#2. Convert the returned JSON object into a Python dictionary.
# jsonify results
ticker_json = r.json()
ticker_header = ticker_json.get('dataset_data').get('column_names')
# extract column names
print('Column Names:%s' %(str(ticker_header)) )

# build dataset as 2D list
ticker_data = ticker_json.get('dataset_data').get('data')
print(ticker_data[0:10])

# extract useful indices
idx_date = ticker_header.index('Date')
idx_open = ticker_header.index('Open')
idx_high = ticker_header.index('High')
idx_low = ticker_header.index('Low')
idx_close = ticker_header.index('Close')
idx_volume = ticker_header.index('Traded Volume')

Column Names:['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']
[['2017-12-29', 51.76, 51.94, 51.45, 51.76, None, 34640.0, 1792304.0, None, None, None], ['2017-12-28', 51.65, 51.82, 51.43, 51.6, None, 40660.0, 2099024.0, None, None, None], ['2017-12-27', 51.45, 51.89, 50.76, 51.82, None, 57452.0, 2957018.0, None, None, None], ['2017-12-22', 51.05, 51.5, 50.92, 51.32, None, 71165.0, 3641949.0, None, None, None], ['2017-12-21', 51.16, 51.52, 50.9, 51.4, None, 120649.0, 6179433.0, None, None, None], ['2017-12-20', 51.88, 52.04, 51.2, 51.27, None, 50587.0, 2610258.0, None, None, None], ['2017-12-19', 52.73, 52.73, 51.07, 51.66, None, 137313.0, 7102361.0, None, None, None], ['2017-12-18', 52.37, 52.75, 51.61, 52.62, None, 129733.0, 6770499.0, None, None, None], ['2017-12-15', 52.7, 52.7, 51.64, 52.01, None, 204080.0, 10596319.0, None, None, None], ['2017-12-14', 53.11, 53.54, 52.15, 52.67, None, 13

In [117]:
#3. Calculate what the highest and lowest opening prices were for the stock in this period.
# vectorize open prices
ticker_open = [x[idx_open] for x in ticker_data]
# create a second vector to clean Nones (first will be used for index)
ticker_open_clean = [x[idx_open] for x in ticker_data if x[idx_open] is not None]
# get max and date
open_max = max(ticker_open_clean)
open_max_date = ticker_data[ticker_open.index(open_max)][idx_date]
# get min and date
open_min = min(ticker_open_clean)
open_min_date = ticker_data[ticker_open.index(open_min)][idx_date]
print("Highest open was $%0.2f on %s" %(open_max, open_max_date))
print("Lowest open was $%0.2f on %s" %(open_min, open_min_date))

Highest open was $53.11 on 2017-12-14
Lowest open was $34.00 on 2017-01-24


In [130]:
#4. What was the largest change in any one day (based on High and Low price)?
# compute absolute delta
abs_delta = [abs(x[idx_high]-x[idx_low]) for x in ticker_data]
# get max delta and date
abs_delta_max = max(abs_delta)
abs_delta_max_date = ticker_data[abs_delta.index(abs_delta_max)][idx_date]
print('The greatest change was $%0.2f on %s' %(abs_delta_max, abs_delta_max_date))

The greatest change was $2.81 on 2017-05-11


In [150]:
# 5. What was the largest change between any two days (based on Closing Price)?
# vectorize closing prices
t0_closing = [x[idx_close] for x in ticker_data]
# copy and shift array
t1_closing = t0_closing.copy()
t1_closing.pop(0)
# calculate daily delta
daily_delta_abs = []
for idx in range(0,len(t1_closing)):
    daily_delta_abs.append( abs( round((t1_closing[idx] - t0_closing[idx]), 2) ) )
# find max daily change and date
daily_delta_abs_max = max(daily_delta_abs)
abs_delta_max_date_t0 = ticker_data[daily_delta_abs.index(daily_delta_abs_max)][idx_date]
abs_delta_max_date_t1 = ticker_data[daily_delta_abs.index(daily_delta_abs_max)+1][idx_date]
print('The greatest daily change was $%0.2f between %s and %s' %(abs_delta_max, abs_delta_max_date_t1, abs_delta_max_date_t0))

The greatest daily change was $2.81 between 2017-08-08 and 2017-08-09


In [166]:
# 6. What was the average daily trading volume during this year?
# vectorize ticker_volume
ticker_volume = [x[idx_volume] for x in ticker_data]
# find max volume and date
volume_max = max(ticker_volume)
volume_max_date = ticker_data[ticker_volume.index(volume_max)][idx_date]
print('The greatest Volume was %0.f on %s' %(volume_max, volume_max_date))

The greatest Volume was 670349 on 2017-03-22


In [169]:
# 7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)
# sort
ticker_volume.sort()
# determine if list is even and find median
if len(ticker_volume)%2:
    # take middle number of sorted list for odd numbered list
    idx_median = int(len(ticker_volume)/2 + 0.5 - 1)
    median = ticker_volume[idx_median]
else:
    # average numbers left and right of the middle
    idx_median = int(len(ticker_volume)/2 - 1)
    median = (ticker_volume[idx_median] + ticker_volume[idx_median+1] ) / 2
print('The median ticker volume is %0.f' %median)

The median ticker volume is 76286
